In [3]:
from elastic_body import *
from utils import *
from progress import ProgressBarText
import time

import numpy as np
import scipy as sp
import scipy.integrate

import sys
import os
import matplotlib.pyplot as plt

res_path = os.path.join(PROJ_PATH, RESULTS_DIR)

In [4]:
# material (moduli)
material_file = open(os.path.join(PROJ_PATH, PARAMS_DIR, 'ps_retarded_nonlin'))
moduli_dict = parse_input_file(material_file)
material_file.close()

# force
impact_dict = {IMPACT_AMPL: 0.02, IMPACT_TIME: 0.5}
ampl = impact_dict[IMPACT_AMPL]
w = impact_dict[IMPACT_TIME]

def f(t):
    if np.abs(t/w) > 20:
        F = 0*t, 0*t, 0*t
    else:
        F = ampl*np.cosh(t/w)**(-2), 0, 0
    return np.asarray(F)[:,None,None]

# simulation times
tmax = 100
dt = 2
t0 = -8*w
T = np.arange(t0, tmax + dt/2, dt)

In [5]:
# output files prefix
output_prefix = 'nonlin_visc_len_200_'

# waveguide
geometry_dict = {BODY_TYPE: BAR,
                 LENGTH: 200,
                 H_Y: 10,
                 H_Z: 10,
                 DOM_NUM: [40, 2, 2],
                 DOM_PNT: [10, 10, 10],
                 DOM_PER: [False,] * 3}

# create body and mesh
params_dict = {**moduli_dict, **geometry_dict, **impact_dict}
body, mesh = create_body_and_mesh(params_dict)

In [9]:
def derivative(t, y):
    bval = [(f(t), 0),] + [(0, 0),] + [(0, 0),]
    vects = decompress_tens_ret(y, mesh, ret_num=body.ret_num)
    ders = body.derivative_nonlin(bval, *vects)
    return compress_tens_ret(*ders)

In [10]:
# create output parameters file
if not os.path.exists(res_path):
    os.mkdir(res_path)
of = open(os.path.join(res_path, output_prefix + PARAMS_FILENAME), 'w')
params_dict[STOP_TIME] = tmax
params_dict[TIME_STEP] = dt
params_dict[START_TIME] = t0
save_params(of, params_dict)

In [11]:
# create arrays
u0 = TensorField(mesh, np.zeros((3,) + mesh.shape))
v0 = TensorField(mesh, np.zeros((3,) + mesh.shape))
r0 = TensorField(mesh, np.zeros((body.ret_num, 3, 3) + mesh.shape))
U = TensorField(mesh, np.zeros((len(T), 3) + mesh.shape))
U[0] = u0

# prepare simulation
integrator = sp.integrate.ode(derivative).set_integrator(
    'dop853', rtol=1e-10, atol=1e-10, nsteps=1e6)
integrator.set_initial_value(compress_tens_ret(u0, v0, r0), t=t0)

# simulate
for k, t in enumerate(ProgressBarText(T[1:]), 1):
    # integrate
    if t > integrator.t:
        integrator.integrate(t)
    vects = decompress_tens_ret(integrator.y, mesh, ret_num=body.ret_num)
    U[k] = vects[0]

Progress: |██████████████████████████████| 100.0% Elapsed: 1:38:40, Estimated: 1:38:40


In [12]:
# save simulation
np.save(os.path.join(res_path, output_prefix + U_FILENAME), U.func)